# 📌Intro.

- 관광 메타데이터의 추가 데이터 수집을 위한 웹 크롤링 진행
- 관광지별 Description과 키워드(Tag) 정보를 수집
- 각 지역별로 관광지명을 검색하여 Descripion과 Tag를 수집
    - Description
        - 1차로 구글맵에서 데이터를 수집(Description) 하고 실패시 네이버 지식백과 활용
        - 네이버 지식백과의 '대한민국 구석구석' 이 한국관광공사와 동일한 데이터이기 때문
        - 한국관광공사의 데이터는 1일 1,000건 API 호출 제한이 있어 전체 데이터를 프로젝트 기한내에 수집이 어려워 우회 방법 사용
    - Tag
        - 카카오맵의 장소별 'Tag' 데이터를 수집

## 크롤링 함수 생성

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from fake_useragent import UserAgent
import chromedriver_autoinstaller
from selenium.webdriver.chrome.service import Service
import edgedriver_autoinstaller
from selenium.webdriver.common.keys import Keys
import time
import re


def crawler(keyword):
    # 웹 브라우징 시 사용되는 User-Agent를 랜덤으로 생성하는 라이브러리
    ua = UserAgent()
    user_agent = ua.random

    # Chrome 웹 브라우저 옵션 설정
    options = webdriver.ChromeOptions()
    options.add_argument('--no-sandbox')  # 보안 기능 비활성화
    options.add_argument('--disable-dev-shm-usage')  # 메모리 사용 최적화
    options.add_argument(f'user-agent={user_agent}')  # fake 봇 만들기
    options.add_argument('--headless')  # 창없는 모드로 실행

    # 로컬에 설치된 Chrome 브라우저의 버전을 확인
    chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]

    try:
        # Chrome 브라우저 드라이버 초기화
        service = Service(f'./{chrome_ver}/chromedriver.exe')
        driver = webdriver.Chrome(service=service, options=options)
    except:
        try:
            # ChromeDriver 자동 설치 및 초기화
            chromedriver_autoinstaller.install(True)
            service = Service(f'./{chrome_ver}/chromedriver.exe')
            driver = webdriver.Chrome(service=service, options=options)
        except:
            # Edge 브라우저 드라이버 초기화
            edgedriver_autoinstaller.install()  # EdgeDriver 자동 설치
            driver = webdriver.Edge()  # EdgeDriver의 경로를 따로 지정하지 않음
    
    # 입력키워드 전처리
    pattern = r'\(.*\)'  # 괄호 안의 텍스트를 찾는 정규표현식 패턴
    re_keyword = re.sub(pattern, '', keyword)
    re_keyword = re_keyword.strip()  # 전처리 후 앞뒤 공백 제거
    

    # 구글 지도에서 개요 정보 수집 시도
    #print(f"구글 지도에서 '{keyword}'에 대한 개요 정보 수집을 시작합니다.")
    driver.get('https://www.google.co.kr/maps/?hl=ko&entry=ttu')
    search_box = WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#searchboxinput')))
    search_box.send_keys(re_keyword)
    search_box.send_keys(Keys.RETURN)
    #timesleep(1)  # 1초 대기

    try:
        overview_tab = WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#QA0Szd > div > div > div.w6VYqd > div:nth-child(2) > div > div.e07Vkf.kA9KIf > div > div > div:nth-child(3) > div > div > button.hh2c6.G7m0Af > div.LRkQ2 > div.Gpq6kf.fontTitleSmall')))
        overview_tab.click()
        overview_text = WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#QA0Szd > div > div > div.w6VYqd > div:nth-child(2) > div > div.e07Vkf.kA9KIf > div > div > div.y0K5Df > div > div.WeS02d.fontBodyMedium > div > div'))).text
        overview_result = "수집성공"
        #print(f"구글 지도에서 '{keyword}'에 대한 개요 정보를 수집했습니다.")
    except:
        #print(f"구글 지도에서 '{keyword}'에 대한 개요 정보 수집에 실패했습니다. 네이버 지식백과로 이동합니다.")
        overview_text = ""
        overview_result = "수집실패"

    # 네이버 지식백과에서 개요 정보 수집 시도
    if overview_result == "수집실패":
        #print(f"네이버 지식백과에서 '{keyword}'에 대한 개요 정보 수집을 시작합니다.")
        driver.get('https://terms.naver.com/')
        ##timesleep(1)  # 1초 대기
        search_box = WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#term_query')))
        search_box.send_keys(keyword)
        search_box.send_keys(Keys.RETURN)
        #timesleep(1)  # 1초 대기

        try:
            target_dictionary = "대한민국 구석구석"
            target_url = None

            for i in range(2, 6):  # 검색 결과 2번째부터 5번째까지 확인
                try:
                    result = WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.CSS_SELECTOR, f'#content > div:nth-child({i}) > ul > li > div.info_area > div.subject > span > a')))
                    if target_dictionary in result.text:
                        target_url = WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.CSS_SELECTOR, f'#content > div:nth-child({i}) > ul > li > div.info_area > div.subject > strong > a'))).get_attribute('href')
                        break
                except:
                    continue

            if target_url:
                driver.get(target_url)
                #timesleep(1)  # 1초 대기
                overview_text = WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#size_ct > p'))).text
                overview_result = "수집성공"
                #print(f"네이버 지식백과에서 '{keyword}'에 대한 개요 정보를 수집했습니다.")
            else:
                #print(f"네이버 지식백과에서 '{keyword}'에 대한 '대한민국 구석구석' 정보를 찾지 못했습니다.")
                overview_text = ""
                overview_result = "수집실패"

        except:
            #print(f"네이버 지식백과에서 '{keyword}'에 대한 개요 정보 수집에 실패했습니다.")
            overview_text = ""
            overview_result = "수집실패"

    # 카카오맵에서 태그 정보 수집 시도
    # 카카오맵에서 태그 정보 수집 시도
    #print(f"카카오맵에서 '{keyword}'에 대한 태그 정보 수집을 시작합니다.")
    driver.get('https://map.kakao.com/?nil_profile=title&nil_src=local')
    #time.sleep(1)  # 1초 대기
    search_box = WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#search\.keyword\.query')))
    search_box.send_keys(keyword)
    search_box.send_keys(Keys.RETURN)  # 엔터 버튼 누르기
    #print("검색어 입력 및 검색 완료")
    #time.sleep(2)  # 2초 대기

    try:
        search_result = WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#info\.search\.place\.list > li:nth-child(1)')))
        #print("검색 결과 찾음")

        try:
            moreview_button = search_result.find_element(By.CSS_SELECTOR, 'div.info_item > div.contact.clickArea > a.moreview')
            #print("상세보기 버튼 찾음")

            # 버튼이 클릭 가능한 상태가 될 때까지 최대 10초 동안 기다림
            WebDriverWait(driver, 1).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'div.info_item > div.contact.clickArea > a.moreview')))

            # JavaScript를 사용하여 버튼 클릭
            driver.execute_script("arguments[0].click();", moreview_button)
            #print("상세보기 버튼 클릭")
            #time.sleep(2)  # 2초 대기

            driver.switch_to.window(driver.window_handles[-1])
            #print("새로운 탭으로 전환")

            try:
                tag_container = WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#mArticle > div.cont_essential > div.details_placeinfo > div:nth-child(4) > div')))
                #print("태그 컨테이너 찾음")
            except:
                tag_container = WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#mArticle > div.cont_essential > div.details_placeinfo > div:nth-child(3) > div')))
                #print("대체 태그 컨테이너 찾음")

            try:
                tags = tag_container.find_elements(By.CSS_SELECTOR, 'div > span > a')
                #print(f"{len(tags)}개의 태그 찾음")
                tag_text = ', '.join([tag.text for tag in tags])
                tag_result = "수집성공"
                #print(f"카카오맵에서 '{keyword}'에 대한 태그 정보를 수집했습니다.")
            except:
                #print("태그를 찾지 못함")
                raise
        except:
            #print("상세보기 버튼을 찾지 못함")
            raise
    except:
        #print(f"카카오맵에서 '{keyword}'에 대한 태그 정보 수집에 실패했습니다.")
        tag_text = ""
        tag_result = "수집실패"

    # 데이터프레임 생성 및 저장
    data = {
        'title': [keyword],
        'overview': [overview_text],
        'tag': [tag_text],
        'overview_result': [overview_result],
        'tag_result': [tag_result]
    }
    df = pd.DataFrame(data)
    #display(df)

    driver.quit()
    #print("크롤링이 완료되었습니다.")
    return df

### 함수 테스트

In [ ]:
keyword = '롯데월드'
crawler(keyword)

## 실행 코드

- 일괄 수집하는 경우 약 2.2만개로 매우 오랜 시간이 걸림
- 지역별로 분산하여 수집하면 속도가 다소 빨라짐
- 이를 위해 코드를 함수화 해서 파일별로 수집
- 작업이 중단 & 재개 되는 것을 고려해 이어서 작업할 경우 기 수집된 데이터에 이어서 수집할 수 있도록 코드 작성

In [ ]:
import os
import gc
import pandas as pd
from tqdm.notebook import tqdm

def crawl_and_save_data(filename_prefix):
    # 크롤링을 시작할 CSV 파일 경로
    csv_path = f'../data/tour_api/{filename_prefix}_filtered_info.csv'
    
    # 크롤링 결과를 저장할 CSV 파일 경로와 파일명
    output_csv_path = f'../data/tour_api/{filename_prefix}_attraction_overview.csv'
    temp_csv_path = f'../data/tour_api/{filename_prefix}_temp_result.csv'  # 중간 결과를 저장할 임시 파일 경로
    
    # CSV 파일 읽어오기
    df = pd.read_csv(csv_path, encoding='utf-8-sig')
    
    # 크롤링 결과를 저장할 데이터프레임 초기화
    if os.path.exists(temp_csv_path):
        result_df = pd.read_csv(temp_csv_path, encoding='utf-8-sig')
        searched_titles = result_df['title'].tolist()
        print(f"임시 파일에서 {len(searched_titles)}개의 데이터를 불러왔습니다.")
    else:
        result_df = pd.DataFrame(columns=['title', 'overview', 'tag', '수집시도결과'])
        searched_titles = []
        print("새로운 임시 파일을 생성합니다.")
    
    # 크롤링 함수를 적용하여 데이터 수집
    start_index = len(searched_titles)
    for index, row in tqdm(df.iloc[start_index:].iterrows(), total=len(df) - start_index, desc='크롤링 진행률'):
        title = row['title']
        crawled_df = crawler(title)
        
        # 크롤링 결과를 데이터프레임에 추가
        result_df = pd.concat([result_df, crawled_df], ignore_index=True)
        
        # 검색한 키워드를 searched_titles 리스트에 추가
        searched_titles.append(title)
        
        # 메모리 관리를 위해 주기적으로 가비지 컬렉션 수행
        if (index - start_index + 1) % 10 == 0:
            gc.collect()
        
        # 10개 단위로 중간 결과 파일 업데이트
        if (index - start_index + 1) % 10 == 0 or index == len(df) - 1:
            result_df.to_csv(temp_csv_path, index=False, encoding='utf-8-sig')
    
    # 최종 결과 파일로 저장
    result_df.to_csv(output_csv_path, index=False, encoding='utf-8-sig')
    
    # 가장 최근에 저장된 파일의 데이터 개수 출력
    if os.path.exists(output_csv_path):
        saved_df = pd.read_csv(output_csv_path, encoding='utf-8-sig')
        print(f"최종적으로 {len(saved_df)}개의 데이터가 {output_csv_path}에 저장되었습니다.")
    else:
        print("저장된 데이터가 없습니다.")
    
    print("크롤링이 완료되었습니다.")

### 제주지역

In [ ]:
# 각 데이터프레임에 대해 함수 호출
crawl_and_save_data('jeju')

### 경기지역

In [ ]:
crawl_and_save_data('gyeonggi')

### 강원 지역

In [ ]:
crawl_and_save_data('gangwon')

### 충북지역

In [ ]:
crawl_and_save_data('chungbuk')

### 충남지역

In [ ]:
crawl_and_save_data('chungnam')

### 경북지역

In [ ]:
crawl_and_save_data('gyeongbuk')

### 경남지역

In [ ]:
crawl_and_save_data('gyeongnam')

### 전북지역

In [ ]:
crawl_and_save_data('jeonbuk')

### 전남지역

In [ ]:
crawl_and_save_data('jeonnam')

### 서울지역

In [ ]:
crawl_and_save_data('seoul')

### 부산지역

In [ ]:
crawl_and_save_data('busan')

### 인천지역

In [ ]:
crawl_and_save_data('incheon')

### 광주광역시 지역

In [ ]:
crawl_and_save_data('gwangju')

### 대전지역

In [ ]:
crawl_and_save_data('daejeon')

### 울산지역

In [ ]:
crawl_and_save_data('ulsan')

### 세종 지역

In [ ]:
crawl_and_save_data('sejong')